In [1]:
import os 
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import json

In [2]:
o = pd.DataFrame()
for file in os.listdir('../ce_data/StatCache'):
    if file[-9:] == "_1024.csv":
        continue
    x = pd.read_csv("../ce_data/StatCache/"+file)
    x['id'] = x.index
    x['id'] = x['id'].apply(lambda x: file[:-4] + "_" + str(x))
    o = pd.concat([x, o], ignore_index=True)

In [3]:
o = o.sort_values("id", ascending=True, ignore_index=True).fillna(0)
print(o)

        random_32    lru_32  random_64    lru_64  random_128   lru_128  \
0        0.000000  0.000032   0.000000  0.000000    0.000000  0.000000   
1        0.000000  0.000066   0.000000  0.000000    0.000000  0.000000   
2        0.000000  0.000000   0.000000  0.000000    0.000000  0.000000   
3        0.000619  0.000413   0.000497  0.000413    0.000436  0.000413   
4        0.001229  0.000826   0.000985  0.000826    0.000924  0.000826   
...           ...       ...        ...       ...         ...       ...   
141556   0.000000  0.000000   0.000000  0.000000    0.000000  0.000000   
141557   0.000000  0.000000   0.000000  0.000000    0.000000  0.000000   
141558   0.000000  0.000000   0.000000  0.000000    0.000000  0.000000   
141559   0.000000  0.000000   0.000000  0.000000    0.000000  0.000000   
141560   0.000000  0.000000   0.000000  0.000000    0.000000  0.000000   

        random_256   lru_256  random_512   lru_512  random_1024  lru_1024  \
0         0.000000  0.000000    0.

In [4]:
cache_misses = pd.read_csv("../ce_data/caches/llc_miss_ratio.csv")
cache_misses = cache_misses.sort_values("id", ignore_index=True)
print(cache_misses)

                           id        32        64       128       256  \
0         400.perlbench-41B_0  1.000000  1.000000  1.000000  0.998652   
1         400.perlbench-41B_1  1.000000  1.000000  1.000000  1.000000   
2        400.perlbench-41B_10  1.000000  0.997375  0.997375  0.997375   
3       400.perlbench-41B_100  0.996656  0.996656  0.996656  0.993311   
4       400.perlbench-41B_101  1.000000  1.000000  1.000000  1.000000   
...                       ...       ...       ...       ...       ...   
141556        657.xz_s-56B_95  1.000000  1.000000  1.000000  1.000000   
141557        657.xz_s-56B_96  1.000000  1.000000  1.000000  1.000000   
141558        657.xz_s-56B_97  1.000000  1.000000  1.000000  1.000000   
141559        657.xz_s-56B_98  1.000000  1.000000  1.000000  1.000000   
141560        657.xz_s-56B_99  1.000000  1.000000  1.000000  1.000000   

             512      1024      2048      4096      8192  
0       0.889488  0.828841  0.803235  0.773585  0.768194  
1    

In [5]:
# Little check
d = {}
for i,j in zip(cache_misses["id"], o["id"]):
    if i != j:
        print(i)
    if i in d: 
        print(i)
    d[i]=1

In [6]:
from sklearn.metrics import mean_squared_error
for i in ['32','64','128','256','512','1024','2048','4096','8192']:
    c_acc_lru = mean_squared_error(o['lru_'+i], cache_misses[i])
    c_acc_rand = mean_squared_error(o['random_'+i], cache_misses[i])
    print(f'Cache size {i}\t: lru_mse = {c_acc_lru},\t random_mse = {c_acc_rand}')

Cache size 32	: lru_mse = 0.7776474799548214,	 random_mse = 0.7688156845581666
Cache size 64	: lru_mse = 0.7791934404196459,	 random_mse = 0.773865207083091
Cache size 128	: lru_mse = 0.7703035776099812,	 random_mse = 0.7683378564248594
Cache size 256	: lru_mse = 0.7515914116312273,	 random_mse = 0.746738916300773
Cache size 512	: lru_mse = 0.6493950752684519,	 random_mse = 0.6474104323601267
Cache size 1024	: lru_mse = 0.3850316586408856,	 random_mse = 0.3850742052080373
Cache size 2048	: lru_mse = 0.3297520903497203,	 random_mse = 0.33225249039596066
Cache size 4096	: lru_mse = 0.27332100119096986,	 random_mse = 0.274371042118911
Cache size 8192	: lru_mse = 0.22277005489409324,	 random_mse = 0.22390588890801244
